<a href="https://colab.research.google.com/github/JeremyPanJD/stuff/blob/master/04_transfer_learning_in_tensorflow_part_1_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section 142. Preparing data for transfer learning model

In [1]:
import zipfile

In [2]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

--2022-06-23 02:29:37--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.194.128, 142.250.4.128, 74.125.200.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.194.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M  55.0MB/s    in 2.9s    

2022-06-23 02:29:40 (55.0 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [3]:
# Unzip the downloaded file
zip_ref = zipfile.ZipFile("/content/10_food_classes_10_percent.zip",'r')
zip_ref.extractall()
zip_ref.close()


In [4]:
import os

# walk through
for dirpath, dirnames, filenames in os.walk("/content/10_food_classes_10_percent"):
  print(f"there are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

there are 2 directories and 0 images in '/content/10_food_classes_10_percent'.
there are 10 directories and 0 images in '/content/10_food_classes_10_percent/test'.
there are 0 directories and 250 images in '/content/10_food_classes_10_percent/test/ice_cream'.
there are 0 directories and 250 images in '/content/10_food_classes_10_percent/test/grilled_salmon'.
there are 0 directories and 250 images in '/content/10_food_classes_10_percent/test/hamburger'.
there are 0 directories and 250 images in '/content/10_food_classes_10_percent/test/pizza'.
there are 0 directories and 250 images in '/content/10_food_classes_10_percent/test/chicken_curry'.
there are 0 directories and 250 images in '/content/10_food_classes_10_percent/test/steak'.
there are 0 directories and 250 images in '/content/10_food_classes_10_percent/test/chicken_wings'.
there are 0 directories and 250 images in '/content/10_food_classes_10_percent/test/sushi'.
there are 0 directories and 250 images in '/content/10_food_classes

Use `imagedatagenerator` class to load in our images in batches

In [5]:
# Setup data inputs
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (224,224)
BATCH_SIZE = 32

train_dir = "/content/10_food_classes_10_percent/train"
test_dir = "/content/10_food_classes_10_percent/test"

train_datagen = ImageDataGenerator (rescale = 1/255.)
test_datagen = ImageDataGenerator (rescale = 1/255.)

print("Training Images:")
train_data_10_percent = train_datagen.flow_from_directory(directory = train_dir,
                                               batch_size = BATCH_SIZE,
                                               target_size = IMAGE_SHAPE,
                                               class_mode = 'categorical')

print("Testing Images:")
test_data = test_datagen.flow_from_directory(directory = test_dir,
                                               batch_size = BATCH_SIZE,
                                               target_size = IMAGE_SHAPE,
                                               class_mode = 'categorical')
                                               

Training Images:
Found 750 images belonging to 10 classes.
Testing Images:
Found 2500 images belonging to 10 classes.


# Section 143: callbacks

* Tracking experiements with TensorBoard callbacks
* Model checkpoint with ModelCheckpoint callback
* Stopping a model from training with the EarlyStopping callback

## Tensorboard callback (functionized because we need to create a new one for each model)

In [ ]:
import datetime #to create timestamp for each model

def create_tensorboard_callback (dir_name, experiment_name):
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
  print(f"Saving TensorBoard log files to {log_dir}")
  return tensorboard_callback

# Section 144: using TensorFlow Hub models

Majority of layers are from TensorFlow Hub


access pretrained models on: https://tfhub.dev/

The following url is copied:
https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1

In [ ]:
%%script false
# Example 
m = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1",
                   trainable=False),  # Can be True, see below.
    tf.keras.layers.Dense(num_classes, activation='softmax') #output layer
])
m.build([None, expect_img_size, expect_img_size, 3])  # Batch input shape.

# Section 145. Building and compiling a TF hub feature extraction model



In [ ]:
# compare the following two models:

resnet_url ='https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/5'

efficientnet_url = 'https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1'

In [ ]:
# Import dependencies

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

In [ ]:
# making a create_model() function to create model from url

def create_model(model_url, num_classes = 10):
  """
  Takes a TensorFlow Hub URL and creates a Keras Sequential Model with it.

  Args:
    model_url (str): A TensorFlow Hub feature extraction URL.
    num_classes (int): Number of output neurons in the output layer,
      should be equal to number of target classes, default 10.
    
  Returns:
    An uncompiled Keras Sequential model with model_url as feature extractor layer 
      and Dense output layer with num_classes output neurons.
  """
  # Download the pretrained model and save it as a Keras layer
  feature_extractor_layer = hub.KerasLayer(model_url, 
                                           trainable = False, # freeze the already learned patterns
                                           name = "feature_extraction_layer",
                                           input_shape = IMAGE_SHAPE + (3,)) 

  # Construct the model 
  model = tf.keras.Sequential([
      feature_extractor_layer,
      layers.Dense(num_classes, activation = "softmax", name = "output_layer")
  ])

  return model

### Creating ResNet TensorFlow Hub Feature extraction model

In [ ]:
# Create Resnet Model
resnet_model = create_model(resnet_url, num_classes = train_data_10_percent.num_classes)

In [ ]:
train_data_10_percent, train_data_10_percent.num_classes

(<keras.preprocessing.image.DirectoryIterator at 0x7f1ffc3c59d0>, 10)

In [ ]:
resnet_model.compile(loss= "categorical_crossentropy",
                     optimizer = tf.keras.optimizers.Adam(),
                     metrics = ["accuracy"])

In [ ]:
resnet_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 feature_extraction_layer (K  (None, 2048)             23564800  
 erasLayer)                                                      
                                                                 
 output_layer (Dense)        (None, 10)                20490     
                                                                 
Total params: 23,585,290
Trainable params: 20,490
Non-trainable params: 23,564,800
_________________________________________________________________


note. only trainable output is the output_layer

In [ ]:
# Fit the model

history_resnet = resnet_model.fit(train_data_10_percent,
                 epochs = 5,
                 verbose = 1,
                 steps_per_epoch = len(train_data_10_percent),
                 validation_data = test_data,
                 validation_steps = len(test_data))

Epoch 1/5
24/24 [==============================] - 450s 19s/step - loss: 1.8240 - accuracy: 0.4013 - val_loss: 1.1578 - val_accuracy: 0.6304
Epoch 2/5
24/24 [==============================] - 440s 19s/step - loss: 0.8798 - accuracy: 0.7280 - val_loss: 0.9064 - val_accuracy: 0.6956
Epoch 3/5
24/24 [==============================] - 441s 19s/step - loss: 0.6283 - accuracy: 0.8213 - val_loss: 0.7603 - val_accuracy: 0.7492
Epoch 4/5
24/24 [==============================] - 439s 19s/step - loss: 0.4670 - accuracy: 0.8853 - val_loss: 0.7059 - val_accuracy: 0.7688
Epoch 5/5
24/24 [==============================] - 435s 19s/step - loss: 0.3764 - accuracy: 0.9080 - val_loss: 0.6781 - val_accuracy: 0.7748


In [ ]:
resnet_model.evaluate(test_data)

79/79 [==============================] - 13s 158ms/step - loss: 0.6265 - accuracy: 0.7960


[0.6264798045158386, 0.7960000038146973]

note. gives the same result as model is not trainable